In [68]:
###최종최종에서 칼럼순서만 변경한 버전######
### key 기반 중복 방지 + 멀티헤더 해제 후 칼럼 추가 ###
#########정말정말 최종본##################
import os, time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoAlertPresentException

DOWNLOAD_DIR = r"C:\Users\mstel\Downloads"
options = Options()
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=options)
driver.get("https://tmacs.kotsa.or.kr/web/TG/TG300/TG3100/Tg2127.jsp?mid=S1810")
wait = WebDriverWait(driver, 10)

# ---------------------- 팝업 / 다운로드 ----------------------
def wait_for_popup(driver, timeout=10):
    WebDriverWait(driver, timeout).until(lambda d: len(d.window_handles) > 1)

def wait_for_download_complete(download_dir=DOWNLOAD_DIR, timeout=30):
    before = set(os.listdir(download_dir))
    end_time = time.time() + timeout
    while time.time() < end_time:
        after = set(os.listdir(download_dir))
        new_files = after - before
        if new_files and any(f.endswith(".xlsx") or f.endswith(".xls") for f in new_files):
            return list(new_files)[0]
        time.sleep(1)
    return None

def handle_structure_download(driver, wait, location_name):
    main_window = driver.current_window_handle
    retries = 0
    while retries < 2:
        try:
            wait_for_popup(driver)
            for handle in driver.window_handles:
                if handle != main_window:
                    driver.switch_to.window(handle)
                    break

            # 상세 탭 클릭
            tab = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#tabId03")))
            tab.click()

            time.sleep(3)

            # 팝업 정보 파싱
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "dl.jogun dd")))
            dd_elements = driver.find_elements(By.CSS_SELECTOR, "dl.jogun dd")

            사고지역 = dd_elements[0].text.strip() if len(dd_elements) > 0 else ""
            사고장소 = dd_elements[1].text.strip() if len(dd_elements) > 1 else ""
            조회기간 = dd_elements[2].text.strip() if len(dd_elements) > 2 else ""

            time.sleep(3)

            # 엑셀 다운로드
            excel_btn = wait.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "#new_popup > div.pop_cont > div.btn_box > a.btn.exbtn")
            ))
            excel_btn.click()

            downloaded_file = wait_for_download_complete()
            if downloaded_file:
                downloaded_path = os.path.join(DOWNLOAD_DIR, downloaded_file)

                # pandas로 멀티헤더 읽기
                df = pd.read_excel(downloaded_path, header=[0, 1, 2])

                # 멀티헤더 → 단일 헤더 변환
                df.columns = ['_'.join([str(c) for c in col if str(c) != 'nan']) for col in df.columns]

                # 맨 앞에 칼럼 추가
                df.insert(0, '사고지역', 사고지역)
                df.insert(1, '사고장소', 사고장소)
                df.insert(2, '조회기간', 조회기간)

                # 저장
                base_filename = "사고분석-지역별.xlsx"
                save_path = os.path.join(DOWNLOAD_DIR, base_filename)
                if os.path.exists(save_path):
                    i = 1
                    while True:
                        numbered_path = os.path.join(DOWNLOAD_DIR, f"사고분석-지역별 ({i}).xlsx")
                        if not os.path.exists(numbered_path):
                            save_path = numbered_path
                            break
                        i += 1

                df.to_excel(save_path, index=False)
                print(f"[✅ 성공] {location_name} → {save_path}")
                os.remove(downloaded_path)
                break
            else:
                print(f"[⚠️ 타임아웃] {location_name}")

        except Exception as e:
            retries += 1
            print(f"[❌ 실패] {location_name} - {e} (재시도 {retries}/2)")
            driver.save_screenshot(f"error_{location_name}.png")
            time.sleep(2)

        finally:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(main_window)

    if retries == 2:
        print(f"[⛔ 완전 실패] {location_name}")

# ---------------------- 중복 방지 visited ----------------------
visited = set()

# ---------------------- 버튼 클릭 ----------------------
def download_page(driver, wait, year_value, sido_text, jijace_text, page_number):
    while True:
        buttons = driver.find_elements(By.CSS_SELECTOR, "#rMateH5__Content69 > div > span > img")
        print(f"📄 {page_number} 페이지, 남은 버튼: {len(buttons)}")

        if not buttons:
            break

        clicked_any = False
        for idx, btn in enumerate(buttons):
            try:
                button_cell = btn.find_element(By.XPATH, "./ancestor::div[contains(@class, 'rMateH5__ImageItemRenderer')]")
                parent_container = button_cell.find_element(By.XPATH, "./parent::div")
                branch_name = parent_container.find_element(By.CSS_SELECTOR, "span.rMateH5__DataGridColumn25").text.strip()
                incident_count = parent_container.find_element(By.CSS_SELECTOR, "span.rMateH5__DataGridColumn27").text.strip()

                key = f"{branch_name}_{incident_count}_{idx}"
                if key in visited:
                    continue

                print(f"👉 클릭: {key}")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
                time.sleep(0.5)
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(1)

                handle_structure_download(driver, wait,
                    f"{year_value}_{sido_text}_{jijace_text}_page{page_number}_{key}")
                visited.add(key)
                time.sleep(1)

                clicked_any = True
                break
            except Exception as e:
                print(f"[❌ 클릭 실패] {e}")
                driver.save_screenshot("click_error.png")

        if not clicked_any:
            print("✅ 이 페이지의 모든 버튼 클릭 완료")
            break

# ---------------------- 스크롤 ----------------------
def scroll_next_page(driver):
    scroll_element = driver.find_element(By.CSS_SELECTOR, ".rMateH5__VBrowserScrollBar")
    current_scroll = driver.execute_script("return arguments[0].scrollTop;", scroll_element)
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + 500;", scroll_element)
    time.sleep(1)
    new_scroll = driver.execute_script("return arguments[0].scrollTop;", scroll_element)
    return current_scroll != new_scroll

# ---------------------- 전체 다운로드 ----------------------
def download_all(driver, wait, year_value, sido_text, jijace_text):
    page = 1
    while True:
        download_page(driver, wait, year_value, sido_text, jijace_text, page)
        if not scroll_next_page(driver):
            print("📌 더 이상 스크롤 불가 → 종료")
            break
        page += 1
    print("✅ 전체 다운로드 완료")

# ---------------------- 실행 ----------------------

test_years = ["2023", "2022"]

for year_value in test_years:
    year_select = Select(driver.find_element(By.ID, "Year"))
    year_select.select_by_value(year_value)

    # 시도 옵션 매번 새로 가져오기
    sido_options = driver.find_elements(By.CSS_SELECTOR, "#sido option")
    for sido_index in range(1, len(sido_options)):  # 0번("선택") 제외
        sido_select = Select(driver.find_element(By.ID, "sido"))
        sido_select.select_by_index(sido_index)
        sido_text = sido_select.first_selected_option.text

        wait.until(EC.presence_of_element_located((By.ID, "jijace")))
        jijace_select = Select(driver.find_element(By.ID, "jijace"))
        jijace_select.select_by_index(0)  # 지자체는 항상 전체(0번)
        jijace_text = jijace_select.first_selected_option.text

        driver.find_element(By.CSS_SELECTOR, "div.btn_wrap > a").click()
        try:
            alert = driver.switch_to.alert
            alert.accept()
        except NoAlertPresentException:
            pass

        time.sleep(2)
        download_all(driver, wait, year_value, sido_text, jijace_text)

driver.quit()


📄 1 페이지, 남은 버튼: 14
👉 클릭: 교보타워사거리_72_0


c:\Users\mstel\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[✅ 성공] 2023_서울_전체_page1_교보타워사거리_72_0 → C:\Users\mstel\Downloads\사고분석-지역별.xlsx
📄 1 페이지, 남은 버튼: 14
👉 클릭: 교보타워사거리_72_1


c:\Users\mstel\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[✅ 성공] 2023_서울_전체_page1_교보타워사거리_72_1 → C:\Users\mstel\Downloads\사고분석-지역별 (1).xlsx
📄 1 페이지, 남은 버튼: 14
👉 클릭: 교보타워사거리_72_2


...

c:\Users\mstel\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[✅ 성공] 2023_세종_전체_page3_고운동799(남동방향 492M)_8_13 → C:\Users\mstel\Downloads\사고분석-지역별 (2128).xlsx
📄 3 페이지, 남은 버튼: 15
👉 클릭: 고운동799(남동방향 492M)_8_14


c:\Users\mstel\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[✅ 성공] 2023_세종_전체_page3_고운동799(남동방향 492M)_8_14 → C:\Users\mstel\Downloads\사고분석-지역별 (2129).xlsx
📄 3 페이지, 남은 버튼: 15
✅ 이 페이지의 모든 버튼 클릭 완료
📌 더 이상 스크롤 불가 → 종료
✅ 전체 다운로드 완료


ElementClickInterceptedException: Message: element click intercepted: Element <a href="#" class="btn btn_primary" onclick="onMove();">...</a> is not clickable at point (935, 33). Other element would receive the click: <a href="/web/TA/member/LoginGuide.jsp?mid=S3111">...</a>
  (Session info: chrome=138.0.7204.184); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementclickinterceptedexception
Stacktrace:
	GetHandleVerifier [0x0x7ff7ba80e415+77285]
	GetHandleVerifier [0x0x7ff7ba80e470+77376]
	(No symbol) [0x0x7ff7ba5d9a6a]
	(No symbol) [0x0x7ff7ba6383e9]
	(No symbol) [0x0x7ff7ba635d8b]
	(No symbol) [0x0x7ff7ba632dd1]
	(No symbol) [0x0x7ff7ba631ce1]
	(No symbol) [0x0x7ff7ba623458]
	(No symbol) [0x0x7ff7ba65860a]
	(No symbol) [0x0x7ff7ba622d06]
	(No symbol) [0x0x7ff7ba658820]
	(No symbol) [0x0x7ff7ba68087f]
	(No symbol) [0x0x7ff7ba6583e3]
	(No symbol) [0x0x7ff7ba621521]
	(No symbol) [0x0x7ff7ba6222b3]
	GetHandleVerif

In [16]:
###오류들이 발생해서 다시 시도시도####
###최종최종에서 칼럼순서만 변경한 버전######
### key 기반 중복 방지 + 멀티헤더 해제 후 칼럼 추가 ###
#########정말정말 최종본##################
import os, time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoAlertPresentException

DOWNLOAD_DIR = r"C:\Users\mstel\Downloads"
options = Options()
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=options)
driver.get("https://tmacs.kotsa.or.kr/web/TG/TG300/TG3100/Tg2127.jsp?mid=S1810")
wait = WebDriverWait(driver, 10)

# ---------------------- 팝업 / 다운로드 ----------------------
def wait_for_popup(driver, timeout=10):
    WebDriverWait(driver, timeout).until(lambda d: len(d.window_handles) > 1)

def wait_for_download_complete(download_dir=DOWNLOAD_DIR, timeout=30):
    before = set(os.listdir(download_dir))
    end_time = time.time() + timeout
    while time.time() < end_time:
        after = set(os.listdir(download_dir))
        new_files = after - before
        if new_files and any(f.endswith(".xlsx") or f.endswith(".xls") for f in new_files):
            return list(new_files)[0]
        time.sleep(1)
    return None

def handle_structure_download(driver, wait, location_name):
    main_window = driver.current_window_handle
    retries = 0
    while retries < 2:
        try:
            wait_for_popup(driver)
            for handle in driver.window_handles:
                if handle != main_window:
                    driver.switch_to.window(handle)
                    break

            # 상세 탭 클릭
            tab = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#tabId03")))
            tab.click()

            time.sleep(3)

            # 팝업 정보 파싱
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "dl.jogun dd")))
            dd_elements = driver.find_elements(By.CSS_SELECTOR, "dl.jogun dd")

            사고지역 = dd_elements[0].text.strip() if len(dd_elements) > 0 else ""
            사고장소 = dd_elements[1].text.strip() if len(dd_elements) > 1 else ""
            조회기간 = dd_elements[2].text.strip() if len(dd_elements) > 2 else ""

            time.sleep(3)

            # 엑셀 다운로드
            excel_btn = wait.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "#new_popup > div.pop_cont > div.btn_box > a.btn.exbtn")
            ))
            excel_btn.click()

            downloaded_file = wait_for_download_complete()
            if downloaded_file:
                downloaded_path = os.path.join(DOWNLOAD_DIR, downloaded_file)

                # pandas로 멀티헤더 읽기
                df = pd.read_excel(downloaded_path, header=[0, 1, 2])

                # 멀티헤더 → 단일 헤더 변환
                df.columns = ['_'.join([str(c) for c in col if str(c) != 'nan']) for col in df.columns]

                # 맨 앞에 칼럼 추가
                df.insert(0, '사고지역', 사고지역)
                df.insert(1, '사고장소', 사고장소)
                df.insert(2, '조회기간', 조회기간)

                # 저장
                base_filename = "사고분석-지역별.xlsx"
                save_path = os.path.join(DOWNLOAD_DIR, base_filename)
                if os.path.exists(save_path):
                    i = 1
                    while True:
                        numbered_path = os.path.join(DOWNLOAD_DIR, f"사고분석-지역별 ({i}).xlsx")
                        if not os.path.exists(numbered_path):
                            save_path = numbered_path
                            break
                        i += 1

                df.to_excel(save_path, index=False)
                print(f"[✅ 성공] {location_name} → {save_path}")
                os.remove(downloaded_path)
                break
            else:
                print(f"[⚠️ 타임아웃] {location_name}")

        except Exception as e:
            retries += 1
            print(f"[❌ 실패] {location_name} - {e} (재시도 {retries}/2)")
            driver.save_screenshot(f"error_{location_name}.png")
            time.sleep(2)

        finally:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(main_window)

    if retries == 2:
        print(f"[⛔ 완전 실패] {location_name}")

# ---------------------- 중복 방지 visited ----------------------
visited = set()

# ---------------------- 버튼 클릭 ----------------------
def download_page(driver, wait, year_value, sido_text, jijace_text, page_number):
    while True:
        buttons = driver.find_elements(By.CSS_SELECTOR, "#rMateH5__Content69 > div > span > img")
        print(f"📄 {page_number} 페이지, 남은 버튼: {len(buttons)}")

        if not buttons:
            break

        clicked_any = False
        for idx, btn in enumerate(buttons):
            try:
                button_cell = btn.find_element(By.XPATH, "./ancestor::div[contains(@class, 'rMateH5__ImageItemRenderer')]")
                parent_container = button_cell.find_element(By.XPATH, "./parent::div")
                branch_name = parent_container.find_element(By.CSS_SELECTOR, "span.rMateH5__DataGridColumn25").text.strip()
                incident_count = parent_container.find_element(By.CSS_SELECTOR, "span.rMateH5__DataGridColumn27").text.strip()

                key = f"{branch_name}_{incident_count}_{idx}"
                if key in visited:
                    continue

                print(f"👉 클릭: {key}")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
                time.sleep(0.5)
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(1)

                handle_structure_download(driver, wait,
                    f"{year_value}_{sido_text}_{jijace_text}_page{page_number}_{key}")
                visited.add(key)
                time.sleep(1)

                clicked_any = True
                break
            except Exception as e:
                print(f"[❌ 클릭 실패] {e}")
                driver.save_screenshot("click_error.png")

        if not clicked_any:
            print("✅ 이 페이지의 모든 버튼 클릭 완료")
            break

# ---------------------- 스크롤 ----------------------
def scroll_next_page(driver):
    scroll_element = driver.find_element(By.CSS_SELECTOR, ".rMateH5__VBrowserScrollBar")
    current_scroll = driver.execute_script("return arguments[0].scrollTop;", scroll_element)
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + 500;", scroll_element)
    time.sleep(1)
    new_scroll = driver.execute_script("return arguments[0].scrollTop;", scroll_element)
    return current_scroll != new_scroll

# ---------------------- 전체 다운로드 ----------------------
def download_all(driver, wait, year_value, sido_text, jijace_text):
    page = 1
    while True:
        download_page(driver, wait, year_value, sido_text, jijace_text, page)
        if not scroll_next_page(driver):
            print("📌 더 이상 스크롤 불가 → 종료")
            break
        page += 1
    print("✅ 전체 다운로드 완료")

# ---------------------- 실행 ----------------------

test_years = ["2023", "2022"]

for year_value in test_years:
    year_select = Select(driver.find_element(By.ID, "Year"))
    year_select.select_by_value(year_value)

    # 광역시/도 (0번 제외)
    sido_options = driver.find_elements(By.CSS_SELECTOR, "#sido option")
    for sido_index in range(1, len(sido_options)):
        sido_select = Select(driver.find_element(By.ID, "sido"))
        sido_select.select_by_index(sido_index)
        sido_text = sido_select.first_selected_option.text

        wait.until(EC.presence_of_element_located((By.ID, "jijace")))
        jijace_options = driver.find_elements(By.CSS_SELECTOR, "#jijace option")

        # 지자체 (0번 제외)
        for jijace_index in range(1, len(jijace_options)):
            jijace_select = Select(driver.find_element(By.ID, "jijace"))
            jijace_select.select_by_index(jijace_index)
            jijace_text = jijace_select.first_selected_option.text

            # 조회 버튼 클릭
            driver.find_element(By.CSS_SELECTOR, "div.btn_wrap > a").click()

            try:
                alert = driver.switch_to.alert
                alert.accept()
            except NoAlertPresentException:
                pass

            time.sleep(2)
            download_all(driver, wait, year_value, sido_text, jijace_text)

driver.quit()




📄 1 페이지, 남은 버튼: 14
👉 클릭: 교보타워사거리_72_0


c:\Users\mstel\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[✅ 성공] 2023_서울_강남구_page1_교보타워사거리_72_0 → C:\Users\mstel\Downloads\사고분석-지역별.xlsx
📄 1 페이지, 남은 버튼: 14
👉 클릭: 교보타워사거리_72_1


c:\Users\mstel\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[✅ 성공] 2023_서울_강남구_page1_교보타워사거리_72_1 → C:\Users\mstel\Downloads\사고분석-지역별 (1).xlsx
📄 1 페이지, 남은 버튼: 14
👉 클릭: 교보타워사거리_72_2


...

c:\Users\mstel\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[✅ 성공] 2023_서울_강동구_page3_합계_624_12 → C:\Users\mstel\Downloads\사고분석-지역별 (83).xlsx
📄 3 페이지, 남은 버튼: 15
👉 클릭: 합계_624_13


c:\Users\mstel\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[✅ 성공] 2023_서울_강동구_page3_합계_624_13 → C:\Users\mstel\Downloads\사고분석-지역별 (84).xlsx
📄 3 페이지, 남은 버튼: 15
👉 클릭: 합계_624_14
[❌ 실패] 2023_서울_강동구_page3_합계_624_14 - Message: 
 (재시도 1/2)
[❌ 실패] 2023_서울_강동구_page3_합계_624_14 - Message: 
 (재시도 2/2)
[⛔ 완전 실패] 2023_서울_강동구_page3_합계_624_14
📄 3 페이지, 남은 버튼: 15
✅ 이 페이지의 모든 버튼 클릭 완료
📌 더 이상 스크롤 불가 → 종료
✅ 전체 다운로드 완료


ElementClickInterceptedException: Message: element click intercepted: Element <a href="#" class="btn btn_primary" onclick="onMove();">...</a> is not clickable at point (935, 33). Other element would receive the click: <a href="/web/TA/member/LoginGuide.jsp?mid=S3111">...</a>
  (Session info: chrome=138.0.7204.184); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementclickinterceptedexception
Stacktrace:
	GetHandleVerifier [0x0x7ff74778e415+77285]
	GetHandleVerifier [0x0x7ff74778e470+77376]
	(No symbol) [0x0x7ff747559a6a]
	(No symbol) [0x0x7ff7475b83e9]
	(No symbol) [0x0x7ff7475b5d8b]
	(No symbol) [0x0x7ff7475b2dd1]
	(No symbol) [0x0x7ff7475b1ce1]
	(No symbol) [0x0x7ff7475a3458]
	(No symbol) [0x0x7ff7475d860a]
	(No symbol) [0x0x7ff7475a2d06]
	(No symbol) [0x0x7ff7475d8820]
	(No symbol) [0x0x7ff74760087f]
	(No symbol) [0x0x7ff7475d83e3]
	(No symbol) [0x0x7ff7475a1521]
	(No symbol) [0x0x7ff7475a22b3]
	GetHandleVerif

In [31]:

##이거는 최종본은 아니고, 시도 잘 돌아가게끔 하는 로직이 추가됨###



import os, time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoAlertPresentException

DOWNLOAD_DIR = r"C:\Users\mstel\Downloads"
options = Options()
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=options)
driver.get("https://tmacs.kotsa.or.kr/web/TG/TG300/TG3100/Tg2127.jsp?mid=S1810")
wait = WebDriverWait(driver, 10)

# ---------------------- 팝업 / 다운로드 ----------------------
def wait_for_download_complete(download_dir=DOWNLOAD_DIR, timeout=30):
    before = set(os.listdir(download_dir))
    end_time = time.time() + timeout
    while time.time() < end_time:
        after = set(os.listdir(download_dir))
        new_files = after - before
        if new_files and any(f.endswith(".xlsx") or f.endswith(".xls") for f in new_files):
            return list(new_files)[0]
        time.sleep(1)
    return None

def handle_structure_download(driver, wait, location_name):
    main_window = driver.current_window_handle
    retries = 0
    while retries < 2:
        try:
            # ✅ 기존 방식으로 팝업 전환
            time.sleep(1)
            for handle in driver.window_handles:
                if handle != main_window:
                    driver.switch_to.window(handle)
                    break

            # 상세 탭 클릭
            tab = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#tabId03")))
            tab.click()
            time.sleep(4)

            # 팝업 정보 파싱
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "dl.jogun dd")))
            dd_elements = driver.find_elements(By.CSS_SELECTOR, "dl.jogun dd")

            사고지역 = dd_elements[0].text.strip() if len(dd_elements) > 0 else ""
            사고장소 = dd_elements[1].text.strip() if len(dd_elements) > 1 else ""
            조회기간 = dd_elements[2].text.strip() if len(dd_elements) > 2 else ""

            time.sleep(3)

            # 엑셀 다운로드
            excel_btn = wait.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "#new_popup > div.pop_cont > div.btn_box > a.btn.exbtn")
            ))
            excel_btn.click()

            downloaded_file = wait_for_download_complete()
            if downloaded_file:
                downloaded_path = os.path.join(DOWNLOAD_DIR, downloaded_file)

                # pandas로 멀티헤더 읽기
                df = pd.read_excel(downloaded_path, header=[0, 1, 2])

                # ✅ 멀티헤더/단일헤더 모두 처리
                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = ['_'.join([str(c) for c in col if str(c) != 'nan']) for col in df.columns]
                else:
                    df.columns = [str(c) for c in df.columns]

                # 맨 앞에 칼럼 추가
                df.insert(0, '사고지역', 사고지역)
                df.insert(1, '사고장소', 사고장소)
                df.insert(2, '조회기간', 조회기간)

                # 저장
                base_filename = "사고분석-지역별.xlsx"
                save_path = os.path.join(DOWNLOAD_DIR, base_filename)
                if os.path.exists(save_path):
                    i = 1
                    while True:
                        numbered_path = os.path.join(DOWNLOAD_DIR, f"사고분석-지역별 ({i}).xlsx")
                        if not os.path.exists(numbered_path):
                            save_path = numbered_path
                            break
                        i += 1

                df.to_excel(save_path, index=False)
                print(f"[✅ 성공] {location_name} → {save_path}")
                os.remove(downloaded_path)
                break
            else:
                print(f"[⚠️ 타임아웃] {location_name}")

        except Exception as e:
            retries += 1
            print(f"[❌ 실패] {location_name} - {e} (재시도 {retries}/2)")
            driver.save_screenshot(f"error_{location_name}.png")
            time.sleep(1)

        finally:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(main_window)

    if retries == 2:
        print(f"[⛔ 완전 실패] {location_name}")

# ---------------------- 중복 방지 visited ----------------------
visited = set()

# ---------------------- 버튼 클릭 ----------------------
def download_page(driver, wait, year_value, sido_text, jijace_text, page_number):
    while True:
        buttons = driver.find_elements(By.CSS_SELECTOR, "#rMateH5__Content69 > div > span > img")
        print(f"📄 {page_number} 페이지, 남은 버튼: {len(buttons)}")

        if not buttons:
            break

        clicked_any = False
        for idx, btn in enumerate(buttons):
            try:
                button_cell = btn.find_element(By.XPATH, "./ancestor::div[contains(@class, 'rMateH5__ImageItemRenderer')]")
                parent_container = button_cell.find_element(By.XPATH, "./parent::div")
                branch_name = parent_container.find_element(By.CSS_SELECTOR, "span.rMateH5__DataGridColumn25").text.strip()
                incident_count = parent_container.find_element(By.CSS_SELECTOR, "span.rMateH5__DataGridColumn27").text.strip()

                key = f"{branch_name}_{incident_count}_{idx}"
                if key in visited:
                    continue

                print(f"👉 클릭: {key}")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
                time.sleep(0.5)
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(1)

                handle_structure_download(driver, wait,
                    f"{year_value}_{sido_text}_{jijace_text}_page{page_number}_{key}")
                visited.add(key)
                time.sleep(1)

                clicked_any = True
                break
            except Exception as e:
                print(f"[❌ 클릭 실패] {e}")
                driver.save_screenshot("click_error.png")
                visited.add(key)  # ✅ 실패한 버튼도 방문 처리

        if not clicked_any:
            print("✅ 이 페이지의 모든 버튼 클릭 완료")
            break

# ---------------------- 스크롤 ----------------------
def scroll_next_page(driver):
    scroll_element = driver.find_element(By.CSS_SELECTOR, ".rMateH5__VBrowserScrollBar")
    current_scroll = driver.execute_script("return arguments[0].scrollTop;", scroll_element)
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + 500;", scroll_element)
    time.sleep(1)
    new_scroll = driver.execute_script("return arguments[0].scrollTop;", scroll_element)
    return current_scroll != new_scroll

# ---------------------- 전체 다운로드 ----------------------
def download_all(driver, wait, year_value, sido_text, jijace_text):
    page = 1
    while True:
        download_page(driver, wait, year_value, sido_text, jijace_text, page)
        if not scroll_next_page(driver):
            print("📌 더 이상 스크롤 불가 → 종료")
            break
        page += 1
    print("✅ 전체 다운로드 완료")

# ---------------------- 실행 ----------------------
test_years = ["2023", "2022"]

for year_value in test_years:
    # 기준년도 선택
    year_select = Select(driver.find_element(By.ID, "Year"))
    year_select.select_by_value(year_value)

    # 광역시/도 옵션 (0번 제외)
    sido_options = driver.find_elements(By.CSS_SELECTOR, "#sido option")
    for sido_index in range(1, len(sido_options)):
        sido_select = Select(driver.find_element(By.ID, "sido"))
        sido_select.select_by_index(sido_index)
        sido_text = sido_select.first_selected_option.text

        wait.until(EC.presence_of_element_located((By.ID, "jijace")))

        # 지자체 옵션 (0번 제외)
        jijace_options = driver.find_elements(By.CSS_SELECTOR, "#jijace option")
        for jijace_index in range(1, len(jijace_options)):
            jijace_select = Select(driver.find_element(By.ID, "jijace"))
            jijace_select.select_by_index(jijace_index)
            jijace_text = jijace_select.first_selected_option.text

            # 조회 버튼 클릭
            driver.find_element(By.CSS_SELECTOR, "div.btn_wrap > a").click()
            try:
                alert = driver.switch_to.alert
                alert.accept()
            except NoAlertPresentException:
                pass

            time.sleep(2)
            download_all(driver, wait, year_value, sido_text, jijace_text)

driver.quit()






📄 1 페이지, 남은 버튼: 14
👉 클릭: 교보타워사거리_72_0


c:\Users\mstel\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[✅ 성공] 2023_서울_강남구_page1_교보타워사거리_72_0 → C:\Users\mstel\Downloads\사고분석-지역별 (46).xlsx
📄 1 페이지, 남은 버튼: 14
👉 클릭: 교보타워사거리_72_1
[❌ 클릭 실패] Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=138.0.7204.184); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff74778e415+77285]
	GetHandleVerifier [0x0x7ff74778e470+77376]
	(No symbol) [0x0x7ff747559a6a]
	(No symbol) [0x0x7ff747545cf5]
	(No symbol) [0x0x7ff74756a7fa]
	(No symbol) [0x0x7ff7475dfc05]
	(No symbol) [0x0x7ff747600192]
	(No symbol) [0x0x7ff7475d83e3]
	(No symbol) [0x0x7ff7475a1521]
	(No symbol) [0x0x7ff7475a22b3]
	GetHandleVerifier [0x0x7ff747a71efd+3107021]
	GetHandleVerifier [0x0x7ff747a6c29d+3083373]
	GetHandleVerifier [0x0x7ff747a8bedd+3213485]
	GetHandleVerifier [0x0x7ff7477a884e+184862]
	GetHandleVe

InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff74778e415+77285]
	GetHandleVerifier [0x0x7ff74778e470+77376]
	(No symbol) [0x0x7ff74755989c]
	(No symbol) [0x0x7ff7475a064f]
	(No symbol) [0x0x7ff7475d84d2]
	(No symbol) [0x0x7ff7475d2fd3]
	(No symbol) [0x0x7ff7475d2099]
	(No symbol) [0x0x7ff747524895]
	GetHandleVerifier [0x0x7ff747a71efd+3107021]
	GetHandleVerifier [0x0x7ff747a6c29d+3083373]
	GetHandleVerifier [0x0x7ff747a8bedd+3213485]
	GetHandleVerifier [0x0x7ff7477a884e+184862]
	GetHandleVerifier [0x0x7ff7477b055f+216879]
	(No symbol) [0x0x7ff747523890]
	GetHandleVerifier [0x0x7ff747ba6558+4370216]
	BaseThreadInitThunk [0x0x7ffb61411fe4+20]
	RtlUserThreadStart [0x0x7ffb6385ef91+33]


In [33]:

############오류들이 발생해서 다시 시도시도####




import os, time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoAlertPresentException

DOWNLOAD_DIR = r"C:\Users\mstel\Downloads"
options = Options()
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=options)
driver.get("https://tmacs.kotsa.or.kr/web/TG/TG300/TG3100/Tg2127.jsp?mid=S1810")
wait = WebDriverWait(driver, 10)

# ---------------------- 팝업 / 다운로드 ----------------------
def wait_for_download_complete(download_dir=DOWNLOAD_DIR, timeout=30):
    before = set(os.listdir(download_dir))
    end_time = time.time() + timeout
    while time.time() < end_time:
        after = set(os.listdir(download_dir))
        new_files = after - before
        if new_files and any(f.endswith(".xlsx") or f.endswith(".xls") for f in new_files):
            return list(new_files)[0]
        time.sleep(1)
    return None

def handle_structure_download(driver, wait, location_name):
    main_window = driver.current_window_handle
    retries = 0
    while retries < 2:
        try:
            time.sleep(1)
            for handle in driver.window_handles:
                if handle != main_window:
                    driver.switch_to.window(handle)
                    break

            tab = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#tabId03")))
            tab.click()
            time.sleep(4)

            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "dl.jogun dd")))
            dd_elements = driver.find_elements(By.CSS_SELECTOR, "dl.jogun dd")

            사고지역 = dd_elements[0].text.strip() if len(dd_elements) > 0 else ""
            사고장소 = dd_elements[1].text.strip() if len(dd_elements) > 1 else ""
            조회기간 = dd_elements[2].text.strip() if len(dd_elements) > 2 else ""

            time.sleep(3)

            excel_btn = wait.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "#new_popup > div.pop_cont > div.btn_box > a.btn.exbtn")
            ))
            excel_btn.click()

            downloaded_file = wait_for_download_complete()
            if downloaded_file:
                downloaded_path = os.path.join(DOWNLOAD_DIR, downloaded_file)

                df = pd.read_excel(downloaded_path, header=[0, 1, 2])

                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = ['_'.join([str(c) for c in col if str(c) != 'nan']) for col in df.columns]
                else:
                    df.columns = [str(c) for c in df.columns]

                df.insert(0, '사고지역', 사고지역)
                df.insert(1, '사고장소', 사고장소)
                df.insert(2, '조회기간', 조회기간)

                base_filename = "사고분석-지역별.xlsx"
                save_path = os.path.join(DOWNLOAD_DIR, base_filename)
                if os.path.exists(save_path):
                    i = 1
                    while True:
                        numbered_path = os.path.join(DOWNLOAD_DIR, f"사고분석-지역별 ({i}).xlsx")
                        if not os.path.exists(numbered_path):
                            save_path = numbered_path
                            break
                        i += 1

                df.to_excel(save_path, index=False)
                print(f"[✅ 성공] {location_name} → {save_path}")
                os.remove(downloaded_path)
                break
            else:
                print(f"[⚠️ 타임아웃] {location_name}")

        except Exception as e:
            retries += 1
            print(f"[❌ 실패] {location_name} - {e} (재시도 {retries}/2)")
            driver.save_screenshot(f"error_{location_name}.png")
            time.sleep(1)

        finally:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(main_window)

    if retries == 2:
        print(f"[⛔ 완전 실패] {location_name}")

# ---------------------- visited ----------------------
visited = set()

# ---------------------- 버튼 클릭 + 내부 스크롤 개선 ----------------------
def download_page(driver, wait, year_value, sido_text, jijace_text, page_number):
    scroll_element = driver.find_element(By.CSS_SELECTOR, ".rMateH5__VBrowserScrollBar")

    while True:
        buttons = driver.find_elements(By.CSS_SELECTOR, "#rMateH5__Content69 > div > span > img")
        if not buttons:
            print("버튼 없음 → 종료")
            break

        for idx, btn in enumerate(buttons):
            try:
                button_cell = btn.find_element(By.XPATH, "./ancestor::div[contains(@class, 'rMateH5__ImageItemRenderer')]")
                parent_container = button_cell.find_element(By.XPATH, "./parent::div")
                branch_name = parent_container.find_element(By.CSS_SELECTOR, "span.rMateH5__DataGridColumn25").text.strip()
                incident_count = parent_container.find_element(By.CSS_SELECTOR, "span.rMateH5__DataGridColumn27").text.strip()

                key = f"{branch_name}_{incident_count}_{idx}"
                if key in visited:
                    continue

                print(f"👉 클릭: {key}")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
                time.sleep(0.5)
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(1)

                handle_structure_download(driver, wait,
                    f"{year_value}_{sido_text}_{jijace_text}_page{page_number}_{key}")
                visited.add(key)
                time.sleep(1)

            except Exception as e:
                print(f"[❌ 클릭 실패] {e}")
                driver.save_screenshot("click_error.png")
                visited.add(key)

        # 스크롤 상태 체크
        current_scroll = driver.execute_script("return arguments[0].scrollTop;", scroll_element)
        max_scroll = driver.execute_script("return arguments[0].scrollHeight - arguments[0].clientHeight;", scroll_element)

        if current_scroll >= max_scroll:
            print("📌 스크롤 끝 → 종료")
            break
        else:
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + 500;", scroll_element)
            time.sleep(1)

    print("✅ 이 페이지의 모든 버튼 다운로드 완료")

# ---------------------- 전체 다운로드 ----------------------
def download_all(driver, wait, year_value, sido_text, jijace_text):
    page = 1
    download_page(driver, wait, year_value, sido_text, jijace_text, page)
    print("✅ 전체 다운로드 완료")

# ---------------------- 실행 ----------------------
test_years = ["2023", "2022"]

for year_value in test_years:
    year_select = Select(driver.find_element(By.ID, "Year"))
    year_select.select_by_value(year_value)

    sido_options = driver.find_elements(By.CSS_SELECTOR, "#sido option")
    for sido_index in range(1, len(sido_options)):
        sido_select = Select(driver.find_element(By.ID, "sido"))
        sido_select.select_by_index(sido_index)
        sido_text = sido_select.first_selected_option.text

        wait.until(EC.presence_of_element_located((By.ID, "jijace")))
        jijace_options = driver.find_elements(By.CSS_SELECTOR, "#jijace option")
        for jijace_index in range(1, len(jijace_options)):
            jijace_select = Select(driver.find_element(By.ID, "jijace"))
            jijace_select.select_by_index(jijace_index)
            jijace_text = jijace_select.first_selected_option.text

            driver.find_element(By.CSS_SELECTOR, "div.btn_wrap > a").click()
            try:
                alert = driver.switch_to.alert
                alert.accept()
            except NoAlertPresentException:
                pass

            time.sleep(2)
            download_all(driver, wait, year_value, sido_text, jijace_text)

driver.quit()



👉 클릭: 교보타워사거리_72_0


c:\Users\mstel\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[✅ 성공] 2023_서울_강남구_page1_교보타워사거리_72_0 → C:\Users\mstel\Downloads\사고분석-지역별 (90).xlsx
👉 클릭: 교보타워사거리_72_1


c:\Users\mstel\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[✅ 성공] 2023_서울_강남구_page1_교보타워사거리_72_1 → C:\Users\mstel\Downloads\사고분석-지역별 (91).xlsx
👉 클릭: 교보타워사거리_72_2


...

c:\Users\mstel\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[✅ 성공] 2023_서울_강남구_page1_합계_1,423_12 → C:\Users\mstel\Downloads\사고분석-지역별 (130).xlsx
👉 클릭: 합계_1,423_13
[❌ 실패] 2023_서울_강남구_page1_합계_1,423_13 - Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff74778e415+77285]
	GetHandleVerifier [0x0x7ff74778e470+77376]
	(No symbol) [0x0x7ff747559a6a]
	(No symbol) [0x0x7ff7475b0406]
	(No symbol) [0x0x7ff7475b06bc]
	(No symbol) [0x0x7ff747603ac7]
	(No symbol) [0x0x7ff7475d864f]
	(No symbol) [0x0x7ff74760087f]
	(No symbol) [0x0x7ff7475d83e3]
	(No symbol) [0x0x7ff7475a1521]
	(No symbol) [0x0x7ff7475a22b3]
	GetHandleVerifier [0x0x7ff747a71efd+3107021]
	GetHandleVerifier [0x0x7ff747a6c29d+3083373]
	GetHandleVerifier [0x0x7ff747a8bedd+3213485]
	GetHandleVerifier [0x0x7ff7477a884e+184862]
	GetHandleVerifier [0x0x7ff7477b055f+216879]
	GetHandleVerifier [0x0x7ff747797084+113236]
	GetHandleVerifier [0x0x7ff747797239+113673]
	GetHandleVerifier [0x0x7ff74777e298+11368]
	BaseThreadInitThunk [0x0x7ffb61411fe4+20]
	RtlUserThreadStart [0x0x7ffb6385ef91+33]
 (재시도 1/2)
[❌ 

c:\Users\mstel\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[✅ 성공] 2023_서울_강남구_page1_합계_1,423_14 → C:\Users\mstel\Downloads\사고분석-지역별 (131).xlsx
📌 스크롤 끝 → 종료
✅ 이 페이지의 모든 버튼 다운로드 완료
✅ 전체 다운로드 완료
👉 클릭: 천호사거리_44_0


KeyboardInterrupt: 

In [5]:
import os, time
# 파일 시간 순으로 정렬
fpath = "./data3"

if os.path.exists(f'{fpath}/사고분석-지역별.xlsx'):
	os.rename(f'{fpath}/사고분석-지역별.xlsx', f'{fpath}/사고분석-지역별 (0).xlsx')

files_with_time = [(f, os.path.getmtime(os.path.join(fpath, f)))
                   for f in os.listdir(fpath) if os.path.isfile(os.path.join(fpath, f))]

files_with_time.sort(key=lambda x: x[1])

sorted_files = [f[0] for f in files_with_time]
print(sorted_files)

['사고분석-지역별 (0).xlsx', '사고분석-지역별 (1).xlsx', '사고분석-지역별 (2).xlsx', '사고분석-지역별 (3).xlsx', '사고분석-지역별 (4).xlsx', '사고분석-지역별 (5).xlsx', '사고분석-지역별 (6).xlsx', '사고분석-지역별 (7).xlsx', '사고분석-지역별 (8).xlsx', '사고분석-지역별 (9).xlsx', '사고분석-지역별 (10).xlsx', '사고분석-지역별 (11).xlsx', '사고분석-지역별 (12).xlsx', '사고분석-지역별 (13).xlsx', '사고분석-지역별 (14).xlsx', '사고분석-지역별 (15).xlsx', '사고분석-지역별 (16).xlsx', '사고분석-지역별 (17).xlsx', '사고분석-지역별 (18).xlsx', '사고분석-지역별 (19).xlsx', '사고분석-지역별 (20).xlsx', '사고분석-지역별 (21).xlsx', '사고분석-지역별 (22).xlsx', '사고분석-지역별 (23).xlsx', '사고분석-지역별 (24).xlsx', '사고분석-지역별 (25).xlsx', '사고분석-지역별 (26).xlsx', '사고분석-지역별 (27).xlsx', '사고분석-지역별 (28).xlsx', '사고분석-지역별 (29).xlsx', '사고분석-지역별 (30).xlsx', '사고분석-지역별 (31).xlsx', '사고분석-지역별 (32).xlsx', '사고분석-지역별 (33).xlsx', '사고분석-지역별 (34).xlsx', '사고분석-지역별 (35).xlsx', '사고분석-지역별 (36).xlsx', '사고분석-지역별 (37).xlsx', '사고분석-지역별 (38).xlsx', '사고분석-지역별 (39).xlsx', '사고분석-지역별 (40).xlsx', '사고분석-지역별 (41).xlsx', '사고분석-지역별 (42).xlsx', '사고분석-지역별 (43).xlsx', '사고분석-지역별 (44).xlsx', '사고분석-지역별 (45).xlsx

In [11]:
import pandas as pd
#파일 합치기
xlsx_files = [f"{fpath}/{f}" for f in sorted_files]

df = pd.read_excel(xlsx_files[0], header=[0])
for f in xlsx_files[1:]:
  tmp = pd.read_excel(f, header=[0])
  df = pd.concat([df, tmp])

df.reset_index(drop=True, inplace=True)

df.columns = ["사고지역", "사고장소", "조회기간", "일련번호", "년도","월","일","요일","시각","사고등급","사망자수","중상자수","경상자수","부상자수","1당 운전자_연령","1당 운전자_성별","1당 운전자_경력","1당 운전자_음주","1당 차량_종별","1당 차량_용도","관련자2","사고형태","기상상태","일광상태","사고유형1","사고유형2","법규위반","도로종류","ACC_NO"]

KeyboardInterrupt: 

In [ ]:
# 결과 확인
print(df.head())
df.tail()

df['길동사거리']']

     사고지역         사고장소                    조회기간  \
0  서울 강남구  역삼동 교보타워사거리  2021.01.01 ~2023.12.31   
1  서울 강남구  역삼동 교보타워사거리  2021.01.01 ~2023.12.31   
2  서울 강남구  역삼동 교보타워사거리  2021.01.01 ~2023.12.31   
3  서울 강남구  역삼동 교보타워사거리  2021.01.01 ~2023.12.31   
4  서울 강남구  역삼동 교보타워사거리  2021.01.01 ~2023.12.31   

   사고구조분석표_일련번호_Unnamed: 0_level_2  사고구조분석표_년도_Unnamed: 1_level_2  \
0                                1                           2021   
1                                2                           2021   
2                                3                           2021   
3                                4                           2021   
4                                5                           2021   

   사고구조분석표_월_Unnamed: 2_level_2  사고구조분석표_일_Unnamed: 3_level_2  \
0                             1                            15   
1                             1                            14   
2                             1                             2   
3                       

KeyError: '길동사거리'

In [13]:
# csv 파일로 내보내기 - 파일명 확인.
df.to_csv("./data3/20222023test.csv", index=False)

In [24]:
import pandas as pd
# ------------- 중복 체크 -------------
df_2223 = pd.read_csv("./data3/test.csv", header=[0])
df_2223['년도']


0       2021
1       2021
2       2021
3       2021
4       2021
        ... 
3749    2023
3750    2023
3751    2023
3752    2023
3753    2023
Name: 년도, Length: 3754, dtype: int64